# Lab 03: Spatial image processing

The purpose of this lab is to practice preprocessing needed on T1 images images and spatial normalization. Although spatial normalization is needed for the functional images as well, we will first get used to the concept with structural images.

## Preparation
* import python tools for plotting 3D images
* copy basic T1 images from our osfshare for easy access to the files for this lab

In [ ]:
%matplotlib inline
from nilearn import plotting
from nilearn import image

In [ ]:
%%bash
# Copy files from lab 01 for easier work with them here
# Name them generic names for ease of seeing how the command line programs work
cp ../01-Lab/osfshare/sub-97-T1w_brain.nii.gz my_T1_brain.nii.gz
cp ../01-Lab/osfshare/sub-97-T1w_defaced.nii.gz my_T1.nii.gz

### How did we get that "brain" image with the skull stripped  again?
* FSL provides a tool called _bet_ which stands for _brain extraction tool_. See the [bet user manual here](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/BET/UserGuide). 
* It requires only the brain image with skull as input and has relatively few options. Performance is generally OK, but it is common to need to tweak the -f and -g options. 
* If you want to get serious about getting a great brain mask (e.g., your brains have many individual differences in shape due to atrophy), you should look to [ANTs](http://stnava.github.io/ANTs/). Here is an [example](https://dpaniukov.github.io/2016/06/06/brain-extraction-with-ants.html) of using their brain extraction tool
* *Good brain extraction is essential to good registration*. If you see poor registration of your images later on then one place to start trouble-shooting is to look at the quality of your brain extraction by comparing the brain mask (what the program thinks is brain) to the image with the skull (did the program include non-brain tissues in the mask OR chop off part of the brain?)
    - Never assume BET worked great, always check!!

In [ ]:
%%bash
# refresher, are my images here?
ls

## T1 image bias correction and image segmentation
* FSL provides a nice tool called [FAST](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FAST) that combines bias field correction (see Handbook p56) and tissue segmentation
* You will need to input a T1 image of the brain without skull, so brain extraction should be run before-hand

In [ ]:
%%bash
# Look at usage
fast

In [ ]:
%%bash
# example usage below
# running this should take ~10 minutes
fast -t 1 -n 3 -g -b -B -v -o my_T1_brain my_T1_brain.nii.gz > fast.log

In [ ]:
%%bash
# get the documentation
fast -h

#### Write out what these options are doing
Verbose output can help with seeing under the hood and tracking the process

-t 1:

-n 3:

-g:

-b:

-B:

-v:

-o:


In [ ]:
%%bash
# Check outputs
ls

### Practice: 
* Use FSLview to open your T1 image with the "seg" images overlayed on top
* What do these images represent?

In [ ]:
%%bash
# Your code

* The bias corrected image ends with "restore" - take a look and compare values in the view and by looking at their histograms

In [ ]:
%%bash
# Your code

## Spatial registration
* Now let's walk through how to spatially align our high-resolution T1 image and our functional bold image to the MNI-152 standard template.

### Open MNI 152 template in FSLview
* Open standard > MNI152_T1_2mm_brain
* Where is origin?
* What does 2mm refer to in the file name?

In [ ]:
%%bash
# Copy variants of MNI to our local directory for easy access
cp /usr/share/fsl/5.0/data/standard/MNI152_T1_2mm_brain.nii.gz ./MNI152_T1_2mm_brain.nii.gz
cp /usr/share/fsl/5.0/data/standard/MNI152_T1_2mm.nii.gz ./MNI152_T1_2mm.nii.gz
cp /usr/share/fsl/5.0/data/standard/MNI152_T1_2mm_brain_mask_dil.nii.gz ./MNI152_T1_2mm_brain_mask_dil.nii.gz
ls

### Preliminary steps
Before registration it is nice if:
* Images are all in same orientation as MNI (such as RPI)
* If you want to do non-linear registration, FSL requires you have both the skulled and brain-extracted image available
* You may get better results if you have bias-field corrected image. We will use our base images for the examples.  

### The core tools in FSL for registration are:
* [FLIRT](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FLIRT) for linear registration problems
    - FLIRT [user guide](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FLIRT/UserGuide#flirt) which includes examples
    - Notice in usage that *estimation* of the transformation and *application* of the transform are separate options
    - FLIRT can also be run with [BBR](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FLIRT_BBR) if your functional image has good gray matter/white matter contrast. BBR best run on the command-line using the epi_reg program.
* [FNIRT](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FNIRT) for non-linear registration 

### Registration of our T1 to the MNI 152 template

#### Step 1: Use FLIRT for estimation of linear transformation between T1 and MNI. Any linear transformation will be represented by a 4 x 4 affine matrix (the files that typically end in .mat)

In [ ]:
%%bash
# Open the GUI (graphical user interface) and click on FLIRT linear registration
# Note the default options

fsl

In [ ]:
%%bash
# Flirt to align T1 to MNI with an affine transformation, with defaults set on command line
# Note the back-slashes (\) are to tell bash to continue the command across a new line, which helps with readability
flirt -in my_T1_brain.nii.gz -ref MNI152_T1_2mm_brain.nii.gz \
-out T1_2_MNI_aff.nii.gz -omat T1_2_MNI.mat \
-cost corratio -dof 12 -searchrx -90 90 -searchry -90 90 -interp trilinear

In [ ]:
%%bash
ls

In [ ]:
%%bash
# We can look at the affine matrix
cat T1_2_MNI.mat

In [ ]:
%%bash
# We can see the separate components of the transformation with avscale
avscale T1_2_MNI.mat

In [ ]:
%%bash
# Let's compare alignment for caudate and gyri
fslview MNI152_T1_2mm_brain.nii.gz T1_2_MNI_aff.nii.gz

#### Step 2: Use FNIRT to estimate a more refined non-linear "warp field" to align structures more locally

In [ ]:
%%bash
fnirt

In [ ]:
%%bash
ls

In [ ]:
%%bash
# FNIRT with default parameters
fnirt --iout=T1_2_MNI_head --in=my_T1 --aff=T1_2_MNI.mat \
--cout=T1_2_MNI_warp --iout=T1_2_MNI --jout=T1_2_MNI_jac \
--config=T1_2_MNI152_2mm \
--ref=MNI152_T1_2mm.nii.gz --refmask=MNI152_T1_2mm_brain_mask_dil.nii.gz \
--warpres=10,10,10

In [ ]:
%%bash
ls

In [ ]:
%%bash
# Again compare alignment for caudate and gyri
fslview MNI152_T1_2mm_brain.nii.gz T1_2_MNI.nii.gz

#### Step 3: Apply these transforms together to your T1 image

In [ ]:
%%bash
applywarp

In [ ]:
%%bash
applywarp -i my_T1_brain.nii.gz -r MNI152_T1_2mm_brain.nii.gz -o T1_2_MNI_nonlin.nii.gz -w T1_2_MNI_warp.nii.gz

In [ ]:
%%bash
# compare the linear registration to the non-linear registration
fslview MNI152_T1_2mm_brain.nii.gz T1_2_MNI_nonlin.nii.gz T1_2_MNI_aff.nii.gz &


#### Nilearn plotting tools allow you to easily plot co-registered images together with "edge" maps

In [ ]:
# Show our MNI as the base and our T1 as edges
mni="MNI152_T1_2mm.nii.gz"
img="T1_2_MNI_nonlin.nii.gz"
display=plotting.plot_anat(mni, display_mode="z",cut_coords=10)
display.add_edges(img)

# Practice problem

Download a new T1 image from OpenfMRI, and complete the following:
* Run brain extraction with bet 
* Run segmentation with FAST
* Register the T1 image to the MNI template using the same strategy as we did above
* Make a plot in your notebook showing the edges of the registered T1 image on top of the MNI standard brain

In [ ]:
%%bash
# We will download the T1 image from the first subject in this [dataset](https://openfmri.org/dataset/ds000101/).
# Code to download new image from Open fMRI
# The image will be named sub-01_T1w.nii.gz 
wget -O sub-01_T1w.nii.gz https://s3.amazonaws.com/openneuro/ds000101/ds000101_R2.0.0/uncompressed/sub-01/anat/sub-01_T1w.nii.gz

In [ ]:
%%bash
# Your code to run bet
# You should NOT need any additional parameters other than input and output, and specifcy that you want a mask
# Name your skull-stripped output sub-01_T1w_brain.nii.gz




In [ ]:
%%bash
# Code to run FAST



In [ ]:
%%bash
# Code to estimate an affine transform
# Name your image sub-01-T1_2_MNI_aff
# Name your affine transform sub-01-T1_2_MNI.mat



In [ ]:
%%bash
# Code to estimate a non-linear transform




In [ ]:
%%bash
# Code to apply transforms to T1
# Name your output sub-01_T1_2_MNI_nonlin.nii.gz


In [ ]:
# Plot your images


